In [ ]:
pip install pandas

In [36]:
import os
import json
import pandas as pd
import traceback

In [37]:
from langchain_openai import ChatOpenAI


In [38]:
from dotenv import load_dotenv

In [39]:
load_dotenv()

True

In [40]:
KEY=os.getenv("OPENAI_API_KEY")


In [22]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [41]:

llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.3)

In [42]:
llm


ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000027FA7B63CE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027FA8720180>, root_client=<openai.OpenAI object at 0x0000027FA7B63A80>, root_async_client=<openai.AsyncOpenAI object at 0x0000027FA7B63E10>, temperature=0.3, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [13]:
%pip install langchain-community
%pip install PyPDF2





Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [43]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_community.callbacks import get_openai_callback
from PyPDF2 import PdfReader
import os
from langchain_core.output_parsers import StrOutputParser

In [44]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} briefly."
)

chain = prompt | llm | StrOutputParser()

In [45]:
llm

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000027FA8720640>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027FA87209D0>, root_client=<openai.OpenAI object at 0x0000027FA87203E0>, root_async_client=<openai.AsyncOpenAI object at 0x0000027FA8720770>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [46]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [47]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [48]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON


"""

In [49]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template=TEMPLATE
    )

In [50]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [51]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [52]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [53]:
review_chain= quiz_evaluation_prompt| llm | StrOutputParser()

In [54]:
generate_evaluate_chain = quiz_chain | review_chain


In [55]:
file_path=r"C:\Users\Disha shetty\MCQgenerator\data.txt"

In [56]:
file_path

'C:\\Users\\Disha shetty\\MCQgenerator\\data.txt'

In [57]:
with open(file_path, 'r') as file:
    TEXT = file.read()


In [58]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[3][4]

From a theoretical viewpoint, probably approximately corr

In [59]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [60]:
NUMBER=5 
SUBJECT="MACHINE LEARNING"
TONE="simple"

In [61]:
from langchain_community.callbacks import get_openai_callback


In [62]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# Quiz prompt
quiz_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template="""
    Create {number} MCQs on {subject}.
    Text: {text}
    Tone: {tone}
    """
)

# Quiz chain
quiz_chain = quiz_prompt | llm | StrOutputParser()

# Review prompt
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["quiz"],
    template="Review the following quiz and suggest improvements:\n{quiz}"
)

# Review chain
review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

# Adapter (VERY IMPORTANT)
adapt_quiz_output = RunnableLambda(lambda quiz: {"quiz": quiz})

# FINAL chain (this is what was missing)
generate_evaluate_chain = quiz_chain | adapt_quiz_output | review_chain


In [71]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE
        }
    )

print(response)
print(cb)





AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-jey3R***************************************FFpj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}